Objective:
$$ Max \sum_{i \in \{1, \ldots, m\}}^{\text{msg}} \sum_{j \in \{1, \ldots, n\}}^{\text{tag}} \sum_{i' \in \{1, \ldots, m\}}^{\text{msg}} y_{iji'} p^{i'}$$

Subject to:

$$ 
\sum_{i \in \{1, \ldots, m\}}^{\text{msg}} k_{ij} - \sum_{i' \in \{1, \ldots, m\}}^{\text{msg}} i' z_{ji'} =0  \quad \forall j \in \{1,...n\}
$$ 

$$
\sum_{i' \in \{0, \ldots, m\}}^{\text{msg}}  z_{ji'} <=1 \quad \forall j \in \{1,...n\}
$$

$$
k_{ij} - z_{ji'} - 2y_{iji'} >=0 \quad \forall i \in \{1,..m\} \quad \forall j \in \{1,..n\} \quad \forall i' \in \{0,..m\}
$$

$$
\sum_{j \in \{1, \ldots, n\}}^{\text{tag}}  k_{ij} >=1 \quad \forall i \in \{1,...m\}
$$

In [1]:
import numpy as np
import pandas as pd
from pulp import *

In [2]:

def max_expected_A(n_tags, m_msgs, p):
   
    # Create a PuLP problem with maximization
    prob = LpProblem("Maximize_Expected_A", LpMaximize)

    # Define the binary variables k indexed by (i, j)
    k = {(i, j): LpVariable(f'k_{i}_{j}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1)}

    # Define the binary variables x indexed by (i, j,i')
    y = {(i, j, m): LpVariable(f'x_{i}_{j}_{m}', cat='Binary') for i in range(1, m_msgs+1) for j in range(1, n_tags+1) for m in range(0, m_msgs+1)}

    
    # Define the continuous variables z indexed by j
    z = {(j,m): LpVariable(f'z_{j}_{m}', cat='Integer', lowBound=0) for j in range(1, n_tags+1) for m in range(0, m_msgs+1)}

    # Constraints
  
    for j in range(1, n_tags+1):
        prob += lpSum(k[i, j] for i in range(1, m_msgs+1)) - lpSum(m*z[j, m] for m in range(1, m_msgs+1)) == 0, f"Sum_k_{j}_equal_z_{j}"

    for j in range(1, n_tags+1):
            prob += lpSum(z[j, m] for m in range(0, m_msgs+1)) <= 1, f"Sum_z_{j}_less_than_1"

    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            for m in range(1, m_msgs+1):
                 prob += k[i,j]+z[j,m] - 2*y[i,j,m] >= 0, f"Constraint_{i}_{j}_{m}"
    
    for j in range(1, n_tags+1):
        prob += lpSum(k[i, j] for i in range(1, m_msgs+1)) >= 1, f"Sum_k_{j}_greater_than_1"


    # Objective Function
    prob += lpSum(y[i,j,m]*P[m] for i in range(1, m_msgs+1) for j in range(1, n_tags+1)for m in range(1, m_msgs+1)), "Maximize_E_A"

    # Solve the problem
    #solver = pulp.PULP_CBC_CMD(timeLimit=120)
    #prob.solve(solver)
    prob.solve()

    # Extract the optimal values of k_{ij}
    k_values = np.zeros((m_msgs, n_tags), dtype=int)
    for i in range(1, m_msgs+1):
        for j in range(1, n_tags+1):
            k_values[i-1, j-1] = value(k[i, j])


    k_df = pd.DataFrame(k_values, columns=[f'k_{j}' for j in range(1, n_tags+1)], index=[f'Msg_{i}' for i in range(1, m_msgs+1)])
    k_df.columns = [f't{i}' for i in range(1,n_tags+1)]
    k_df.index = [f'm{i}' for i in range(1,m_msgs+1)]


    sum_row = k_df.sum()
    k_df.loc['Sum'] = sum_row


    # Print the variable values
    for v in prob.variables():
        print(f"{v.name} = {v.varValue}")

    # Print the objective value
    print(f"Objective value: {value(prob.objective)}")


    return k_df


In [3]:
n_tags = 5
m_msgs = 10
p = 0.8
P = {i: p**(i) for i in range(0, m_msgs+2)}  # p^i coefficients 


In [4]:
max_expected_A(n_tags, m_msgs, p)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/3698344b56a0450aa7e9998b451386ea-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/3698344b56a0450aa7e9998b451386ea-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 520 COLUMNS
At line 3936 RHS
At line 4452 BOUNDS
At line 5058 ENDATA
Problem MODEL has 515 rows, 605 columns and 1705 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 91.9469 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 505 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 500 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 500 strengthened rows, 0 su

,t1,t2,t3,t4,t5
m1,0,0,1,0,1
m2,1,0,0,1,0
m3,0,1,1,1,1
m4,0,1,1,1,0
m5,0,0,0,1,1
m6,1,1,1,0,0
m7,0,0,0,1,0
m8,1,0,0,0,1
m9,0,0,0,0,1
m10,1,1,1,0,0


In [5]:
k_df

NameError: name 'k_df' is not defined

In [ ]:
n_tags = 4
n_msgs = 8
p = 0.7
P = {i: p**(i) for i in range(1, n_msgs+2)}  # p^i coefficients
M = 1000     # A sufficiently large number for the constraints

In [ ]:
optimal_k = max_expected_A(n_tags, n_msgs, p)

Welcome to the CBC MILP Solver 
Version: 2.10.3 
Build Date: Dec 15 2019 

command line - /Library/Frameworks/Python.framework/Versions/3.10/lib/python3.10/site-packages/pulp/solverdir/cbc/osx/64/cbc /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/cd41318c3d8244ed89083fabe429a57e-pulp.mps -max -timeMode elapsed -branch -printingOptions all -solution /var/folders/jv/12n64fl91y1d9jr6561fpx2c0000gn/T/cd41318c3d8244ed89083fabe429a57e-pulp.sol (default strategy 1)
At line 2 NAME          MODEL
At line 3 ROWS
At line 917 COLUMNS
At line 4066 RHS
At line 4979 BOUNDS
At line 5336 ENDATA
Problem MODEL has 912 rows, 356 columns and 2180 elements
Coin0008I MODEL read with 0 errors
Option for timeMode changed from cpu to elapsed
Continuous objective value is 15.6473 - 0.00 seconds
Cgl0003I 0 fixed, 0 tightened bounds, 252 strengthened rows, 128 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 479 strengthened rows, 0 substitutions
Cgl0003I 0 fixed, 0 tightened bounds, 484 strengthened rows, 0 

,t1,t2,t3,t4
m1,0,1,1,0
m2,0,0,0,0
m3,0,0,0,1
m4,1,0,1,1
m5,1,1,1,0
m6,1,0,0,0
m7,0,0,0,1
m8,0,1,0,0
Sum,3,3,3,3


In [6]:
k = np.array([[0,1,1,0],
[0,0,0,0],
[0,0,0,1],
[1,0,1,1],
[1,1,1,0],
[1,0,0,0],
[0,0,0,1],
[0,1,0,0]])